In [1]:
# Dependências
import requests
import zipfile
import io
import pandas as pd
import unicodedata
import re

#### Dados Votação

In [2]:
# df_votos = pd.read_csv('Bases/votacao_secao_2022_RN.csv', sep=';', encoding='cp1252')

In [3]:
# Download do arquivo ZIP em memória
response = requests.get("https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_RN.zip" )
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_file = [file for file in z.namelist() if file.endswith(".csv")][0]
        with z.open(csv_file) as f:
            df_votos = pd.read_csv(f, encoding="ISO-8859-1", sep=";")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {response.status_code}")

In [4]:
df_votos.drop(df_votos.loc[df_votos['DS_CARGO']!='DEPUTADO FEDERAL'].index, inplace=True)
df_votos.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
9,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,24,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,3,1155,200001608805,ESCOLA MUNICIPAL COMERCIAL PROFESSORA GERALDA ...,"RUA JORGE CAMINHA, 118"
10,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,114,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,14,1341,200001608805,ESCOLA ESTADUAL JOSE QUERINO,"POVOADO DE PASSAGEM DOS CABOCLOS, S/N"
11,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,119,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,85,1147,200001608805,ESCOLA MUNICIPAL LUIZ LIBERALINO DE CARVALHO,"POVOADO BOM JESUS, S/N"
12,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,152,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,1,1023,200001608805,ESCOLA ESTADUAL PROFA. MARIA LIDIA DA SILVA,"RUA THEODORICO BEZERRA,132"
13,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,131,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,1,1074,200001608805,ESCOLA ESTADUAL PROFA CALPURNIA C DE AMORIM,"RUA MANOEL GONCALVES DE MELO, 42"


#### Dados Censitários - Por Distritos

In [5]:
'''
DPOs = Domicílios Particulares Ocupados
'''

df_distritos = pd.read_csv('Bases/Distritos.csv').rename(columns={'v0001': 'Pessoas',
                                                                  'v0002': 'Domicílios',
                                                                  'v0003': 'Domicílios Particulares',
                                                                  'v0004': 'Domicílios Coletivos',
                                                                  'v0005': 'Média de moradores em DPOs',
                                                                  'v0006': 'Percentual de DPOs Imputados',
                                                                  'v0007': 'Domicílios Particulares Ocupados'
                                                                  })
df_distritos.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,4750,4747,3,2.768747,0.996329,3814
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,24138,24103,35,2.932901,1.890418,19255
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,4339,4334,5,3.211385,0.177883,3373
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,1199,1199,0,2.972755,0.201816,991
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,6043,6040,3,2.811687,0.392319,4843


In [6]:
df_renda = pd.read_csv('Bases/Rendimento.csv').rename(columns={'Total': 'Total Indivíduos'})
df_renda.head()

,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,Total,2675748,288918,717278,323928,175559,60684,22246,7715,1078875,24
1,Acari,9537,1024,3101,1001,478,115,31,7,3779,2400109
2,Açu,44769,5554,13101,4715,2331,625,167,53,18208,2400208
3,Afonso Bezerra,8927,1789,2442,656,211,24,11,3,3791,2400307
4,Água Nova,2533,333,662,160,63,6,0,0,1309,2400406


In [7]:
df_censitario = pd.merge(df_distritos, df_renda, left_on='NM_DIST', right_on='Distrito', how='inner')
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,...,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,...,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,...,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,...,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,...,11487,2191,3412,807,363,78,16,6,4614,2400505


In [8]:
'''
Normalizando os nomes dos distritos para eles baterem com df_votos
'''

def normalize_text(value):
    if isinstance(value, str):
        # Removendo acentos e caracteres especiais
        value = ''.join(
            char for char in unicodedata.normalize('NFD', value)
            if unicodedata.category(char) != 'Mn'
        )
        # Convertendo para letra maiúscula
        value = value.upper()
        # Removendo caracteres não-alfanuméricos exceto espaços
        value = re.sub(r'[^A-Z0-9 ]', '', value)
    return value

# Aplicando a função nas colunas
df_censitario['NM_MUN'] = df_censitario['NM_MUN'].apply(normalize_text)
df_censitario['NM_DIST'] = df_censitario['NM_DIST'].apply(normalize_text)
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,ACARI,ACARI,28,10597,...,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,ACU,ACU,104,56496,...,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,AFONSO BEZERRA,AFONSO BEZERRA,25,10839,...,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,AGUA NOVA,AGUA NOVA,4,2946,...,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,ALEXANDRIA,ALEXANDRIA,22,13640,...,11487,2191,3412,807,363,78,16,6,4614,2400505


#### Dados Unificados

In [9]:
df_unificado = pd.merge(df_votos, df_censitario, left_on='NM_MUNICIPIO', right_on='NM_DIST', how='inner')
df_unificado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,21514,2118,5343,2759,1575,376,55,8,9276,2401107
1,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,11873,2175,3384,598,230,46,9,5,5426,2411403
2,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,695027,32863,165649,115979,73350,32817,14692,5692,253853,2408102
3,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,8365,1154,2473,761,216,55,18,6,3682,2407500
4,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,...,23678,2364,7629,2585,1603,460,128,39,8869,2409407
